<h1>Video Companion Guide Challenge</h1>
<h2>Submission by: Adam Łucek</h2>
<h4>Automating the creation and media population of markdown companion guides with ✨AI✨</h4>
<div>
<img src="tweet.png" width="500"/>
</div>
<hr></hr>


### Dependencies
---

In [35]:
from pytube import YouTube as pyt
from moviepy.editor import *
import os
from moviepy.editor import AudioFileClip
from openai import OpenAI
import time
import json
import re
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import cv2

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg" 

### Time Keeping Function
---

In [22]:
def start_timer():
    global start_time
    start_time = time.time()

def show_current_runtime():
        return round(time.time() - start_time, 2)

### Downloading the Audio & Video
---

In [36]:
def download_video(url, path):
    yt = pyt(url)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path=f"{path}/original_files/video", filename="video_file.mp4")
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_stream.download(output_path=f"{path}/original_files/audio", filename="audio_file.mp4")

def get_title(url):
    yt = pyt(url)
    return yt.title

### Checking Audio File Size, and Chunking it if Large
---

In [37]:
def split_audio(file_path, chunk_size_mb=12, output_folder="split_chunks"):
    global split_audio_return
    
    if not os.path.exists(file_path):
        print(f"File {file_path} not found.")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"File size: {file_size_mb:.2f} MB")

    if file_size_mb <= chunk_size_mb:
        print("File size is within the limit. No need to split.")
        return
    else:
        split_audio_return = True

    clip = AudioFileClip(file_path)
    total_duration = clip.duration
    chunk_duration = (chunk_size_mb / file_size_mb) * total_duration

    # Split the audio
    start = 0
    part = 1
    while start < total_duration:
        end = min(start + chunk_duration, total_duration)
        chunk = clip.subclip(start, end)
        chunk_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(file_path))[0]}_part{part}.mp4")
        chunk.write_audiofile(chunk_filename, bitrate="64k", codec="aac")

        print(f"Created chunk: {chunk_filename}")

        start = end
        part += 1

    clip.close()

### Transcribing with Whisper-1 & Writing to JSON File(s)
---

In [38]:
def create_json(split_audio_return, input_folder, output_folder="transcript_json"):
    if not os.path.exists(input_folder):
        print(f"The folder {input_folder} does not exist.")
        return
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    client = OpenAI()

    if not split_audio_return:
        # If audio is not split, use the path to the original file
        original_audio_path = f"{input_folder}/original_files/audio"        
        if os.path.exists(original_audio_path):
            for filename in os.listdir(original_audio_path):
                if filename.endswith(".mp4"):
                    process_audio_file(client, original_audio_path, filename, output_folder)
        else:
            print(f"The original audio folder {original_audio_path} does not exist.")
    else:
        input_folder_chunks = f'{path}/split_chunks'
        # If audio is split, iterate over the split audio files
        for filename in os.listdir(f'{path}/split_chunks'):
            if filename.endswith(".mp4"):
                process_audio_file(client, input_folder_chunks, filename, output_folder)

def process_audio_file(client, folder_path, filename, output_folder):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )

        json_filename = f"{os.path.splitext(filename)[0]}_transcript.json"
        output_path = os.path.join(output_folder, json_filename)

        with open(output_path, 'w') as f:
            json.dump(transcript.segments, f, indent=4)

        print(f"Transcript for {filename} saved to {output_path}")

### Cleaning & Concatenating the Transcription JSON files
---

In [39]:
def extract_part_number(filename):
    match = re.search(r'part(\d+)', filename)
    return int(match.group(1)) if match else 0

def process_file(filepath, max_id, last_end_time):
    with open(filepath, 'r') as file:
        data = json.load(file)
        time_adjustment = last_end_time - float(data[0]['start']) if last_end_time else 0
        new_data = []
        for entry in data:
            new_entry = {
                'id': max_id + 1,
                'start': float(entry['start']) + time_adjustment,
                'end': float(entry['end']) + time_adjustment,
                'text': entry['text']
            }
            new_data.append(new_entry)
            max_id += 1
        return new_data, max_id, new_data[-1]['end'] if new_data else last_end_time

def process_transcripts(directory_path, split_audio_return):
    files = os.listdir(directory_path)
    combined_data = []
    max_id = -1
    last_end_time = 0.0

    if split_audio_return:
        sorted_files = sorted(
            [file for file in files if file.startswith('audio') and file.endswith('.json')],
            key=extract_part_number
        )
    else:
        sorted_files = [file for file in files if file.endswith('.json') and not 'part' in file]

    for filename in sorted_files:
        full_path = os.path.join(directory_path, filename)
        processed_data, max_id, last_end_time = process_file(full_path, max_id, last_end_time)
        combined_data.extend(processed_data)

    # Output the combined data to a new JSON file
    output_path = os.path.join(directory_path, 'combined_data.json')
    with open(output_path, 'w') as outfile:
        json.dump(combined_data, outfile, indent=4)



### Also Grabbing the Full Transcript
---

In [40]:
def full_transcript(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    transcript = ""
    for entry in data:
        transcript += entry['text']
    return transcript

### Function to Combine Short Chunks of Transcription
---

In [41]:
def merge_short_documents(documents, min_length=2000):
    i = 0
    while i < len(documents) - 1:
        current_doc = documents[i]
        if len(current_doc.page_content) < min_length:
            documents[i + 1].page_content = current_doc.page_content + documents[i + 1].page_content
            del documents[i]
        else:
            i += 1
    return documents

### Main Markdown File Prompting with GPT-4-Turbo & LangChain Setup
---

In [42]:
guide_prompt_template = """

Below is a script from a video that I am making into a companion guide blog post first. \
You are a helpful assistant made to assist in the creation I'm doing. \
This is a continuation of a guide so include chapters, key summaries, and incorporate visual aids and direct links to relevant parts of the video, \
however do not include any conclusion or overarching title. \
For visual aids, specific frames from the video will be identified where images can be inserted to enhance understanding. \
For direct links, portions of the text should be hyperlinked to their corresponding times in the video. \
To indicate that a sentence should be hyperlinked, insert the raw text of the transcript next to the word with the indicator <HYPERLINK: "corresponding transcript text">. \
To indicate a picture regarding the text, insert the indicator <PICTURE: "corresponding transcript text">. \
It is crucial to use the raw text from the transcript that will be used, as the additional tools that will be inserting the hyperlinks and pictures need this to know where in the video to look.

In this blog post, in addition to the paragraphs: \

Create titles or headings that encapsulate main points and ideas \

Format your response in markdown, ensuring distinction and clean styling between titles and paragraphs. \
Be sure to include the image placeholders, and hyperlinks with enough distinguishable text WITHOUT ANY QUOTATIONS, as the placeholders will be fed into a semantic search algorithm. \
This structured approach will be applied to the entire transcript. \
The example below only shows one style, but use multiple styles including different headings, bullet points, and other markdown elements when needed. \

Here are shortened example of the input and shortened expected output:

example input:

Hi everyone. So in this video I'd like us to cover the process of tokenization in large language models. Now you see here that I have a sad face and that's because tokenization is my least favorite part of working with large language models but unfortunately it is necessary to understand in some detail because it is fairly hairy, gnarly and there's a lot of hidden foot gums to be aware of and a lot of oddness with large language models typically traces back to tokenization. So what is tokenization? Now in my previous video Let's Build GPT from Scratch we actually already did tokenization but we did a very naive simple version of tokenization. So when you go to the Google Colab for that video you see here that we loaded our training set and our training set was this Shakespeare dataset. Now in the beginning the Shakespeare dataset is just a large string in Python it's just text and so the question is how do we plug text into large language models and in this case here we created a vocabulary of 65 possible characters that we saw occur in this string. These were the possible characters and we saw that there are 65 of them and then we created a lookup table for converting from every possible character a little string piece into a token an integer. So here for example we tokenized the string hi there and we received this sequence of tokens and here we took the first 1000 characters of our dataset and we encoded it into tokens and because this is character level we received 1000 tokens in a sequence so token 18, 47, etc. Now later we saw that the way we plug these tokens into the language model is by using an embedding table and so basically if we have 65 possible tokens then this embedding table is going to have 65 rows and roughly speaking we're taking the integer associated with every single token we're using that as a lookup into this table and we're plucking out the corresponding row and this row is trainable parameters that we're going to train using backpropagation and this is the vector that then feeds into the transformer and that's how the transformer sort of perceives every single token. So here we had a very naive tokenization process that was a character level tokenizer

example output:

Introduction to Tokenization
----------------------------

Welcome to our comprehensive guide on tokenization in large language models (LLMs). Tokenization is a critical yet complex aspect of working with LLMs, essential for understanding how these models process text data. Despite its challenges, tokenization is foundational, as it converts strings of text into sequences of tokens, small units of text that LLMs can manage more effectively.

<PICTURE: Now you see here that I have a sad face and that's because tokenization is my least favorite part of working with large language models but unfortunately it is necessary to understand in some detail because it is fairly hairy, gnarly and there's a lot of hidden foot gums>

Understanding the Basics of Tokenization
----------------------------------------

Tokenization involves creating a vocabulary from all unique characters or words in a dataset and converting each into a corresponding integer token. This process was briefly introduced in our "Let's Build GPT from Scratch" video, where we tokenized a Shakespeare dataset at a character level, creating a vocabulary of 65 possible characters.

<HYPERLINK: So what is tokenization? Now in my previous video Let's Build GPT from Scratch we actually already did tokenization but we did a very naive simple version of tokenization. So when you go to the Google Colab for that video you see here that we loaded>

The Role of Embedding Tables in Tokenization
--------------------------------------------

After tokenization, the next step involves using an embedding table, where each token's integer is used as a lookup to extract a row of trainable parameters. These parameters, once trained, feed into the transformer model, allowing it to perceive each token effectively.

<PICTURE: using backpropagation and this is the vector that then feeds into the transformer and that's how the transformer sort of perceives every single token. So here we had a very naive tokenization process that was a character level tokenizer>

end examples.

Here is the transcript:

{transcript}

"""

output_parser = StrOutputParser()
llm = ChatOpenAI(temperature=0.0, model="gpt-4-turbo-preview")
guide_prompt = ChatPromptTemplate.from_template(guide_prompt_template)

guide_chain = (
    {"transcript": RunnablePassthrough()} 
    | guide_prompt
    | llm
    | output_parser
)

def generate_markdown(merged_docs, path, guide_chain):
    markdown_outputs = []
    for doc in merged_docs:
        output = guide_chain.invoke(doc.page_content)
        markdown_outputs.append(output)
    combined_output = '\n\n'.join(markdown_outputs)
    with open(f'{path}/transcript_json/llm_outline.txt', 'w') as file:
        file.write(combined_output)

### Dealing with the Placeholders, Grabbing Pictures & Formatting Hyperlinks
---

In [43]:
def grab_frame(video, second):
    frames_dir = 'frames'
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)
    cap = cv2.VideoCapture(video)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = round(int(second * fps))
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number >= total_frames:
        print(f"Error: Frame number {frame_number} exceeds total frames in video.")
        cap.release()
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        cap.release()
        return None

    frame_path = os.path.join(frames_dir, f'frame_{second}.jpg')
    cv2.imwrite(frame_path, frame)
    cap.release()

    return frame_path

def retrieve_time(segment):
    docs = retriever.get_relevant_documents(segment)
    docs_dict = json.loads(docs[0].page_content)
    start_time = docs_dict["start"]
    end_time = docs_dict["end"]
    time = (start_time + end_time) / 2
    final_time = round(time)
    return final_time

def create_hyperlink(segment, url):
    time = retrieve_time(segment)
    time_link = f"{url}&t={time}s"
    return time_link

def format_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def process_placeholder(placeholder):
    if placeholder.startswith("<PICTURE:"):
        description = placeholder[9:-1]
        time = retrieve_time(description)
        image_path = grab_frame(video_path, time)
        # Embed the image using markdown with a specified width
        return f'<img src="{image_path}" alt="{description}" width="450"/>'
    elif placeholder.startswith("<HYPERLINK:"):
        text = placeholder[11:-1]
        time = retrieve_time(text)
        formatted_time = format_seconds_to_hms(time)
        hyperlink = create_hyperlink(text, url)
        return f'[Jump to this part of the video: {formatted_time}]({hyperlink})'
    else:
        return placeholder

def replace_placeholders(content):
    placeholders = re.findall(r"<[^>]+>", content)
    for placeholder in placeholders:
        replacement = process_placeholder(placeholder)
        content = content.replace(placeholder, replacement, 1)
    return content

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def convert_txt(path, title, db):
    txt_file_path = f'{path}/transcript_json/llm_outline.txt'
    output_file_path = f'{path}/companion_guide.txt'
    global video_path
    video_path = f'{path}/original_files/video/video_file.mp4'
    global retriever
    retriever = db.as_retriever(search_kwargs={"k": 1})
    
    content = read_file(txt_file_path)
    updated_content = replace_placeholders(content)
    
    with open(output_file_path, 'w') as file:
        file.write(updated_content)
    
    print(f"Updated markdown content has been written to {output_file_path}")

### Main Script
---

In [32]:
# main script
start_timer()
# URL & Path of Interest
url = 'https://www.youtube.com/watch?v=zduSFxRajkE'
path = '/Users/adamlucek/Documents/Jupyter/karpathy_guide_challenge'

print(f"Downloading Video & Audio, Runtime: {show_current_runtime()}")
# download video, audio, and details
download_video(url, path)
title = get_title(url)
print(f"Video & Audio Downloaded, Runtime: {show_current_runtime()}")

print(f"Checking File Size & Splitting if Necessary, Runtime: {show_current_runtime()}")
# Check filesize, split into multiple files if needed
split_audio_return = False
split_audio(f"{path}/original_files/audio/audio_file.mp4")
print(f"Audio Checked & Split, Runtime: {show_current_runtime()}")

print(f"Processing Audio File with Whisper-1, Runtime: {show_current_runtime()}")
# Process audio files with Whisper and create JSON files of output
create_json(split_audio_return, path)
print(f"Audio Processed with Whisper-1, Runtime: {show_current_runtime()}")

print(f"Cleaning Data, Runtime: {show_current_runtime()}")
# Combine if needed, clean extra data
process_transcripts(f"{path}/transcript_json", split_audio_return)
print(f"Data Cleaned, Runtime: {show_current_runtime()}")

print(f"Pulling Full Transcript, Runtime: {show_current_runtime()}")
# Pull the full transcript
video_transcript = full_transcript(f'{path}/transcript_json/combined_data.json')
print(f"Transcript Pulled, Runtime: {show_current_runtime()}")

print(f"Chunking & Splitting Transcript, Runtime: {show_current_runtime()}")
# Embed and chunk transcript
text_splitter = SemanticChunker(OpenAIEmbeddings())
split_docs = text_splitter.create_documents([video_transcript])
merged_docs = merge_short_documents(split_docs)
print(f"Transcript Chunked, Runtime: {show_current_runtime()}")

print(f"Embedding Transcript, Runtime: {show_current_runtime()}")
# Embed documents
json_loader = JSONLoader(f"{path}/transcript_json/combined_data.json", jq_schema=".[]", text_content=False)
json_texts = json_loader.load()
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(json_texts, embeddings)
print(f"Transcript Embedded, Runtime: {show_current_runtime()}")

print(f"Generating Markdown Outline with GPT-4-T, Runtime: {show_current_runtime()}")
# Generate markdown of file with GPT-4-T
generate_markdown(merged_docs, path, guide_chain)
print(f"Markdown File Generated, Runtime: {show_current_runtime()}")

print(f"Replacing Placeholders With Pictures & Links, Runtime: {show_current_runtime()}")
# Replace placeholders with hyperlinks and pictures
convert_txt(path, title, db)
print(f"Report Finished, Runtime: {show_current_runtime()}")

Video & Audio Downloaded, Runtime: 11.26
Checking File Size & Splitting if Necessary, Runtime: 11.26
File size: 46.61 MB
MoviePy - Writing audio in split_chunks/audio_file_part1.mp4


MoviePy - Done.
Created chunk: split_chunks/audio_file_part1.mp4
MoviePy - Writing audio in split_chunks/audio_file_part2.mp4


MoviePy - Done.
Created chunk: split_chunks/audio_file_part2.mp4
MoviePy - Writing audio in split_chunks/audio_file_part3.mp4


MoviePy - Done.
Created chunk: split_chunks/audio_file_part3.mp4
MoviePy - Writing audio in split_chunks/audio_file_part4.mp4


MoviePy - Done.
Created chunk: split_chunks/audio_file_part4.mp4
Audio Checked & Split, Runtime: 110.94
Processing Audio File with Whisper-1, Runtime: 110.94
Transcript for audio_file_part4.mp4 saved to transcript_json/audio_file_part4_transcript.json
Transcript for audio_file_part3.mp4 saved to transcript_json/audio_file_part3_transcript.json
Transcript for audio_file_part2.mp4 saved to transcript_json/audio_file_part2_transcript.json
Transcript for audio_file_part1.mp4 saved to transcript_json/audio_file_part1_transcript.json
Audio Processed with Whisper-1, Runtime: 474.22
Cleaning Data, Runtime: 474.22
Data Cleaned, Runtime: 474.24
Pulling Full Transcript, Runtime: 474.24
Transcript Pulled, Runtime: 474.24
Chunking & Splitting Transcript, Runtime: 474.24
Transcript Chunked, Runtime: 477.68
Embedding Transcript, Runtime: 477.68
Transcript Embedded, Runtime: 482.44
Generating Markdown Outline with GPT-4-T, Runtime: 482.44
Markdown File Generated, Runtime: 1278.96
Replacing Placeholder

## **Notes**
#### **Cost:**  ~$1.91 for this pass

* \$1.10 from GPT\-4\-Turbo
  * 56,546 Context Tokens \& 17,710 Generated Tokens
* \$0.80 from Whisper\-1
  * 8,016 seconds transcribed
* \$0.02 from text\-embedding\-002\-v2
  * 172,561 Context Tokens
  
#### **Time from URL to Full Report:** 21 minutes, 56 Seconds

### **Limitations: (aka my todo list)** 
* vector search can sometimes be innacurate, point towards wrong part of the video out of order with this methods.
* Inconsistency of number of pictures and hyperlinks across the board, as it's done automatically
* Unnable to input entire script at once (believe me I tried), so all the downsides that come with sequential processing
* Could be prettier 
* My own lack of programming knowledge


# Program Output
---

Introduction to Tokenization in Large Language Models
------------------------------------------------------

Welcome to our exploration of tokenization in large language models (LLMs). Tokenization is a crucial, albeit complex, process that plays a foundational role in how LLMs understand and process text data. Despite its challenges, mastering tokenization is essential for anyone working with LLMs.

<img src="frames/frame_7.jpg" alt=" Now you see here that I have a sad face and that's because tokenization is my least favorite part" width="450"/>

### What is Tokenization?

Tokenization is the process of converting strings of text into sequences of tokens, which are smaller, manageable units of text for LLMs. This process involves creating a vocabulary of all unique characters or words in a dataset and assigning each a unique integer token.

[Jump to this part of the video: 00:00:27](https://www.youtube.com/watch?v=zduSFxRajkE&t=27s)

### The Role of Embedding Tables

After tokenization, embedding tables come into play. These tables use the integer associated with each token as a lookup to extract a row of trainable parameters. These parameters, once trained, are what the transformer model uses to "understand" each token.

<img src="frames/frame_139.jpg" alt=" So here we had a very naive tokenization process that was a character level tokenizer" width="450"/>

### Beyond Character-Level Tokenization

While the initial example was a simple character-level tokenizer, state-of-the-art language models use more sophisticated methods for constructing token vocabularies. One such method is byte pair encoding (BPE), which we will delve into, demonstrating its application and significance in tokenization for LLMs.

### Byte Pair Encoding (BPE)

BPE is a method used to construct character chunks for token vocabularies, moving beyond simple character-level tokenization. This algorithm plays a crucial role in how modern LLMs, like GPT-2, handle tokenization, allowing for a more efficient and nuanced understanding of text.

[Jump to this part of the video: 00:02:55](https://www.youtube.com/watch?v=zduSFxRajkE&t=175s)

### Tokenization's Impact on Language Model Performance

Tokenization is not just a technical detail; it significantly impacts the performance of LLMs. Issues such as difficulty with spelling tasks, simple string processing, handling non-English languages, and even simple arithmetic can often be traced back to tokenization strategies.

### Exploring Tokenization with Live Examples

To better understand tokenization's effects, we explore live examples using a web app that allows for real-time tokenization in your browser. This demonstration highlights how tokenization can vary significantly based on the context, language, and even formatting of the text.

<img src="frames/frame_365.jpg" alt=" So I have it loaded here and what I like about this web app is that tokenization is running a sort of live in your browser in JavaScript" width="450"/>

### Tokenization and Non-English Languages

The tokenization process can disproportionately affect non-English languages, often resulting in a higher number of tokens for the same content when compared to English. This discrepancy can lead to inefficiencies and limitations in how LLMs process and understand non-English text.

### The Challenge of Tokenizing Code

Tokenization also presents unique challenges when dealing with code, such as Python. The way spaces and indentation are handled can significantly impact the model's ability to understand and generate code, highlighting the importance of considering tokenization in the context of programming languages.

### Conclusion

Understanding tokenization is essential for working with large language models. By exploring its complexities, impacts, and the methods used to optimize it, such as byte pair encoding, we gain a deeper insight into the inner workings of LLMs and how they process text data.

Optimizing Token Space in Language Models
-----------------------------------------

In the quest for efficiency within large language models (LLMs), the management of token space emerges as a critical consideration. The transition from GPT-2 to GPT-4 tokenizers showcases a significant leap towards optimizing this space, reducing the token count for identical strings by approximately half.

### The Impact of Tokenizer Choice on Token Count

When comparing tokenizers, a striking difference in efficiency is observed. The GPT-2 tokenizer, for instance, generates a token count of 300 for a specific string. Switching to the GPT-4 tokenizer, labeled as CL 100k base, the token count dramatically drops to 185 for the same string. This reduction is attributed to the GPT-4 tokenizer's larger vocabulary size, approximately doubling that of its predecessor, GPT-2.

[Jump to this part of the video: 01:13:18](https://www.youtube.com/watch?v=zduSFxRajkE&t=4398s)

### Achieving a Denser Input for the Transformer

The densification of input to the transformer is a direct consequence of this reduction in token count. With fewer tokens representing the same amount of text, the transformer can now process a broader context within the same computational constraints. This efficiency is crucial for enhancing the model's ability to predict subsequent tokens based on a more extensive textual context.

### Balancing the Token Vocabulary

However, the expansion of the token vocabulary is not without its challenges. As the number of tokens increases, so does the size of the embedding table and the complexity of the output prediction mechanism, particularly the softmax layer. This necessitates a careful balance, finding a "sweet spot" in the vocabulary size that ensures both density and efficiency in processing.

### Enhanced Handling of White Space in Python Code

A noteworthy improvement in the GPT-4 tokenizer is its handling of white spaces in Python code. By grouping multiple spaces into a single token, the tokenizer significantly enhances the efficiency of representing Python code. This design choice by OpenAI not only densifies the input but also improves the model's performance in code prediction tasks.

<img src="frames/frame_849.jpg" alt=" You see that here these four spaces are represented as one single token for the three spaces here and then the token spaces and here seven spaces were all grouped into a single token" width="450"/>

This strategic grouping of white spaces into fewer tokens exemplifies the thoughtful design considerations that contribute to the overall improvement in LLMs from GPT-2 to GPT-4, beyond mere architectural and optimization tweaks.

### Conclusion

The evolution of tokenizers from GPT-2 to GPT-4 highlights a significant stride towards optimizing token space in LLMs. By efficiently managing token count and improving the handling of specific data types like Python code, these advancements promise a more effective and nuanced approach to text processing and prediction within LLMs.

## Tokenization and Encoding in Language Models
### Understanding the Need for Tokenization

Tokenization is the process of converting strings of text into integers within a fixed vocabulary, which are then used to make lookups into a table of vectors. These vectors are fed into the transformer as input. This process becomes complex when we consider supporting multiple languages and special characters, such as emojis.

<img src="frames/frame_901.jpg" alt=" So remember what we want to do. We want to take strings and feed them into language models" width="450"/>

### The Challenge of Supporting Multiple Languages and Characters

The complexity of tokenization arises from the need to support a wide range of characters found across different languages and on the internet. Unicode code points serve as a universal standard, defining roughly 150,000 characters across 161 scripts. However, directly using Unicode code points for tokenization is not practical due to the vast vocabulary size and the evolving nature of the Unicode standard.

[Jump to this part of the video: 00:15:53](https://www.youtube.com/watch?v=zduSFxRajkE&t=953s)

### Exploring Unicode and Encodings

Unicode code points are a way to represent characters from various languages and scripts. Python's `ORD` function can be used to find the code point of a character. However, the direct use of Unicode code points is inefficient due to the large vocabulary size. Instead, encodings like UTF-8, UTF-16, and UTF-32 are used to translate Unicode text into binary data or byte streams, with UTF-8 being the most common due to its variable length encoding and compatibility with ASCII.

<img src="frames/frame_961.jpg" alt=" So Unicode code points are defined by the Unicode consortium as part of the Unicode standard" width="450"/>

### The Role of UTF-8 Encoding

UTF-8 encoding translates Unicode code points into a byte stream, which can range from one to four bytes per code point. This encoding is preferred for its efficiency and compatibility with ASCII. However, using UTF-8 naively would result in a small vocabulary size and inefficiently long sequences for language models.

[Jump to this part of the video: 00:18:42](https://www.youtube.com/watch?v=zduSFxRajkE&t=1122s)

### Byte Pair Encoding (BPE) Algorithm

To address the limitations of using raw UTF-8 byte streams, the Byte Pair Encoding (BPE) algorithm is employed. BPE compresses byte sequences by iteratively finding and replacing the most frequent pairs of bytes with a new token, thereby reducing the sequence length while expanding the vocabulary in a controlled manner. This method allows for efficient encoding of text into tokens that can be used by language models.

<img src="frames/frame_1431.jpg" alt=" So as I mentioned the byte pair encoding algorithm is not all that complicated" width="450"/>

### Implementing Byte Pair Encoding

The implementation of BPE involves encoding text into UTF-8, converting bytes to integers, and then applying the BPE algorithm to iteratively compress the byte sequences. This process involves identifying the most common pairs of bytes, replacing them with a new token, and repeating the process to gradually reduce the sequence length and adjust the vocabulary size.

[Jump to this part of the video: 00:31:07](https://www.youtube.com/watch?v=zduSFxRajkE&t=1867s)

### Practical Application of BPE

A practical example of applying BPE is demonstrated by encoding a paragraph into UTF-8, identifying the most common byte pairs, and iteratively merging them to compress the sequence. This process is repeated until a desired vocabulary size or sequence length is achieved, showcasing the effectiveness of BPE in preparing text for language models.

<img src="frames/frame_1625.jpg" alt=" So here's what i did i went to this blog post that i enjoyed and i took the first paragraph" width="450"/>

### Conclusion

The process of tokenization, particularly through the use of the Byte Pair Encoding algorithm, is essential for efficiently preparing text for language models. By understanding and implementing BPE, developers can effectively manage the challenges of supporting multiple languages and characters, ensuring that language models can process text data effectively.

[Jump to this part of the video: 00:34:21](https://www.youtube.com/watch?v=zduSFxRajkE&t=2061s)

Optimizing Vocabulary Size for Efficiency
-----------------------------------------

Finding the ideal vocabulary size is crucial for the efficiency of tokenization in large language models. A larger vocabulary can reduce the sequence length but requires careful tuning to find the optimal balance. GPT-4, for example, utilizes a vocabulary of approximately 100,000 tokens, demonstrating the scale at which modern models operate.

<img src="frames/frame_2073.jpg" alt=" The more steps we take, the larger will be our vocabulary, and the shorter will be our sequence" width="450"/>

Advanced Tokenization Techniques
--------------------------------

### Iterative Byte-Pair Encoding

To refine our tokenizer, we employ an iterative process known as Byte-Pair Encoding (BPE), which allows for more representative statistics and sensible results by analyzing longer texts. This method involves encoding text into bytes, then iteratively merging the most common byte pairs to reduce the overall sequence length while expanding the vocabulary.

[Jump to this part of the video: 00:35:09](https://www.youtube.com/watch?v=zduSFxRajkE&t=2109s)

### Setting the Final Vocabulary Size

Determining the final vocabulary size is a critical step in the tokenization process, acting as a hyperparameter that influences the tokenizer's performance. For our example, we aim for a vocabulary size of 276 tokens, achieved through 20 specific merges, starting from an initial set of 256 byte tokens.

<img src="frames/frame_2170.jpg" alt=" So let's say for us, we're going to use 276 because that way we're going to be doing exactly 20 merges" width="450"/>

### The Process of Merging Tokens

The merging process involves identifying the most common pairs of tokens and combining them into a new token, effectively reducing the total number of tokens while maintaining the ability to represent the original text. This process is repeated iteratively, with each new token becoming eligible for further merging, thereby creating a more efficient encoding scheme.

[Jump to this part of the video: 00:37:46](https://www.youtube.com/watch?v=zduSFxRajkE&t=2266s)

Evaluating Compression Efficiency
---------------------------------

After completing the specified number of merges, it's essential to assess the tokenizer's efficiency by examining the compression ratio achieved. This ratio indicates how effectively the tokenizer can reduce the size of the original text, with a higher ratio signifying greater efficiency. In our example, a compression ratio of approximately 1.27 was achieved after 20 merges.

<img src="frames/frame_2324.jpg" alt=" One thing we can take a look at as well is we can take a look at the compression ratio that we've achieved" width="450"/>

Training the Tokenizer: A Separate Preprocessing Stage
------------------------------------------------------

### The Role of the Tokenizer

The tokenizer serves as a distinct preprocessing stage, separate from the training of the large language model itself. It is responsible for converting raw text into a sequence of tokens and vice versa, facilitating the model's ability to process and understand the text.

[Jump to this part of the video: 00:10:06](https://www.youtube.com/watch?v=zduSFxRajkE&t=606s)

### Importance of a Dedicated Training Set

For optimal performance, the tokenizer may require a different training set than the large language model, encompassing a variety of languages and data types. This diversity ensures that the tokenizer can effectively handle different kinds of text, resulting in more efficient tokenization and, consequently, better model performance.

<img src="frames/frame_2503.jpg" alt=" So for example, when you're training the tokenizer, as I mentioned, we don't just care about the performance of English text" width="450"/>

Encoding and Decoding with the Trained Tokenizer
-------------------------------------------------

### The Process of Decoding

Decoding is the process of converting a sequence of tokens back into the original text. This step is crucial for understanding the output of the language model and for further processing or analysis of the generated text.

[Jump to this part of the video: 00:42:45](https://www.youtube.com/watch?v=zduSFxRajkE&t=2565s)

## Crafting a Custom Vocabulary for Tokenization
### Understanding the Vocabulary Creation Process

Creating a custom vocabulary is a pivotal step in the tokenization process for large language models. This involves mapping token IDs to their corresponding byte objects, starting with the raw bytes for tokens from 0 to 255 and then incorporating all merges in a sorted manner. This method ensures a comprehensive bytes representation of tokens, crucial for effective tokenization.

<img src="frames/frame_2932.jpg" alt=" So there are many different ways to do it.Here's one way" width="450"/>

### The Importance of Python Version in Tokenization

It's essential to note the significance of using modern Python versions, specifically Python 3.7 and above, for tokenization tasks. These versions guarantee the order of items in a dictionary, which is critical for the correct iteration and insertion of elements into the merges dictionary during the vocabulary creation process.

[Jump to this part of the video: 00:44:16](https://www.youtube.com/watch?v=zduSFxRajkE&t=2656s)

## Handling Token Encoding and Decoding
### Converting Token IDs to Text

The process of converting token IDs back into readable text involves looking up their byte representations in the vocabulary, concatenating these bytes, and then decoding them back into Python strings using UTF-8. This step is crucial for transforming the raw bytes into a format that can be easily understood and utilized.

<img src="frames/frame_2686.jpg" alt=" And then here, this is one way in Python to concatenate all these bytes together to create our tokens" width="450"/>

### Addressing UTF-8 Decoding Errors

A common challenge in tokenization is handling invalid UTF-8 byte sequences, which can result in decoding errors. To overcome this, it's recommended to use the `errors=replace` option in the `bytes.decode` function, which substitutes invalid byte sequences with a special replacement character. This approach ensures that the decoding process is robust and can handle any anomalies in the token IDs.

[Jump to this part of the video: 00:47:11](https://www.youtube.com/watch?v=zduSFxRajkE&t=2831s)

## Implementing Token Encoding
### Encoding Strings into Token IDs

The final step in the tokenization process is encoding strings back into token IDs. This involves creating a function that takes a string as input and outputs a list of integers representing the token IDs. This function is essential for converting text data into a format that can be processed by large language models.

<img src="frames/frame_2907.jpg" alt=" So we are going to implement this error right here, where we are going to be given a string, and we want to encode it into tokens" width="450"/>

Advanced Tokenization Techniques
--------------------------------

In this section, we delve into a more sophisticated approach to tokenization, moving beyond the basics to explore how encoding and merging strategies can enhance the process for large language models.

### Encoding Text into UTF-8 Bytes

The initial step in this advanced tokenization process involves encoding the text into UTF-8, converting it into a series of raw bytes. This method provides a foundation by transforming the text into a more manageable form for further processing.

<img src="frames/frame_2940.jpg" alt=" So this is one of the ways that I came up with. So the first thing we're going to do is we are going to take our text, encode it into UTF-8 to get the raw bytes." width="450"/>

### Utilizing a Merges Dictionary

Following the encoding, the process utilizes a merges dictionary to identify and combine specific byte pairs. This dictionary is crucial for determining which bytes can be merged based on their occurrence and relevance within the text.

[Jump to this part of the video: 00:49:23](https://www.youtube.com/watch?v=zduSFxRajkE&t=2963s)

### Sequential Merging Based on Precedence

An essential aspect of this tokenization method is the sequential merging of bytes, adhering to a predefined order. This ensures that merges are performed logically, respecting the hierarchical structure of the merges dictionary.

[Jump to this part of the video: 00:49:33](https://www.youtube.com/watch?v=zduSFxRajkE&t=2973s)

### Identifying Merge Candidates

To efficiently merge byte pairs, the process involves identifying potential candidates for merging. This is achieved by analyzing the sequence of tokens and utilizing statistical methods to pinpoint pairs that are eligible for merging.

[Jump to this part of the video: 00:50:44](https://www.youtube.com/watch?v=zduSFxRajkE&t=3044s)

### Implementing a Min Function for Optimal Merging

A sophisticated technique employed in this process is the use of a `min` function to determine the most suitable pair for merging. This approach ensures that the merging sequence is optimized, focusing on pairs that align with the merges dictionary's order.

[Jump to this part of the video: 00:51:16](https://www.youtube.com/watch?v=zduSFxRajkE&t=3076s)

### Handling Non-Mergable Pairs

An important consideration in this tokenization method is the handling of byte pairs that cannot be merged. By assigning an infinite value to non-mergable pairs, the process effectively excludes them from the merging candidates, ensuring a smooth and logical merging sequence.

[Jump to this part of the video: 00:52:46](https://www.youtube.com/watch?v=zduSFxRajkE&t=3166s)

### Potential Challenges and Solutions

While this advanced tokenization technique offers significant benefits, it's crucial to be aware of potential challenges. One such challenge is the possibility of the merging function failing under certain conditions. Understanding and addressing these challenges is key to implementing a robust tokenization process.

[Jump to this part of the video: 00:53:05](https://www.youtube.com/watch?v=zduSFxRajkE&t=3185s)

Understanding the Merging Process in Tokenization
-------------------------------------------------

In the journey of tokenization, a critical step involves identifying and merging pairs of tokens based on specific criteria. This process is essential for reducing the complexity of the data and making it more manageable for language models.

### The Challenge of Finding Mergeable Pairs

The algorithm seeks pairs that can be merged, but it encounters a significant hurdle when no pairs meet the merging criteria. This situation is indicated by all pairs evaluating to `float infs` for the merging criterion, leading to a scenario where the algorithm cannot proceed with any merges.

<img src="frames/frame_3193.jpg" alt=" If there's nothing to merge, then there's nothing in merges that is satisfied anymore" width="450"/>

### Breaking Out When No Merges Are Possible

A crucial part of the process is identifying when no further merges can be made. This is detected when a pair, not in the `merges` dictionary, becomes the first element in `stats` by default, signaling that merging cannot continue. At this point, the algorithm will break out of the loop, concluding that no additional pairs can be merged.

[Jump to this part of the video: 00:53:49](https://www.youtube.com/watch?v=zduSFxRajkE&t=3229s)

### Implementing the Merge

When a mergeable pair is found, the algorithm proceeds to merge it. This involves looking up the pair in the `mergers` dictionary to find its index and then replacing every occurrence of the pair in the tokens list with this index. This step is repeated until no more pairs can be merged, effectively simplifying the token list.

<img src="frames/frame_3261.jpg" alt=" So we're going to look into the mergers dictionary for that pair to look up the index" width="450"/>

### Handling Special Cases

The implementation also needs to account for special cases, such as when the token list contains a single character or is empty. In these scenarios, the `stats` list would be empty, causing issues with the merging process. To address this, the algorithm includes a condition to return early if the length of the tokens is less than two, thereby avoiding errors related to empty or singular token lists.

[Jump to this part of the video: 00:55:23](https://www.youtube.com/watch?v=zduSFxRajkE&t=3323s)

By understanding these steps and challenges, we gain insight into the complexities of the tokenization process and the intricate details involved in preparing data for language models.

Exploring Tokenization and Its Complexities
-------------------------------------------

In this section, we delve into the intricacies of tokenization, a fundamental process in the operation of large language models (LLMs). Tokenization is not just about converting text into tokens; it's about understanding the nuances that come with encoding and decoding text, ensuring the integrity of the data throughout the process.

### The Basics of Encoding and Decoding

Tokenization involves two primary processes: encoding and decoding. Encoding transforms a string of text into a sequence of tokens, while decoding attempts to revert tokens back to the original text. However, it's crucial to note that decoding doesn't always guarantee a perfect return to the original string due to the limitations in token sequences being valid UTF-8 byte streams.

<img src="frames/frame_3346.jpg" alt=" Okay, and then second, I have a few test cases here for us as well" width="450"/>

### Byte Pair Encoding: A Closer Look

Byte Pair Encoding (BPE) is a tokenization strategy that builds a dictionary of merges from a training set, creating a binary forest atop raw bytes. This method allows for efficient encoding and decoding between raw text and token sequences, laying the groundwork for more advanced tokenization techniques used in state-of-the-art LLMs.

[Jump to this part of the video: 00:56:57](https://www.youtube.com/watch?v=zduSFxRajkE&t=3417s)

### Advancing to State-of-the-Art Tokenizers

As we transition from basic tokenization methods to those employed by leading-edge LLMs, the complexity of the process increases significantly. We begin this exploration by examining the GPT series, specifically focusing on the tokenizer used in GPT-2.

#### GPT-2 and Byte-Pair Encoding

GPT-2 utilizes the byte-pair encoding algorithm, but with a twist. It applies a more sophisticated approach to avoid unnecessary merges, such as combining frequently occurring words with punctuation. This method aims to separate semantics from punctuation, enhancing the model's understanding of text.

<img src="frames/frame_3459.jpg" alt=" So let's kick things off by looking at the GPT series" width="450"/>

#### Enforcing Merging Rules

To refine the tokenization process, GPT-2 introduces manual rules to prevent certain types of characters from merging. This top-down approach ensures that the tokenizer maintains a clear distinction between different elements of the text, optimizing the model's performance.

[Jump to this part of the video: 00:59:11](https://www.youtube.com/watch?v=zduSFxRajkE&t=3551s)

### GPT-2 Tokenizer Implementation

A deep dive into GPT-2's tokenizer reveals the intricacies of its implementation. Despite being named `encoder.py`, this script encompasses both encoding and decoding functionalities, highlighting the tokenizer's critical role in the model's architecture.

[Jump to this part of the video: 00:59:29](https://www.youtube.com/watch?v=zduSFxRajkE&t=3569s)

This exploration of tokenization, from its basic principles to its application in cutting-edge LLMs like GPT-2, underscores the importance of this process in the development and operation of language models. By understanding and refining tokenization, researchers and developers can enhance the efficiency and effectiveness of LLMs.

Exploring Advanced Tokenization Techniques
------------------------------------------

Tokenization is a fundamental step in preparing text for processing by large language models (LLMs). It involves breaking down text into manageable pieces, known as tokens, which the model can interpret. This section delves into the complexities of tokenization, highlighting the use of regular expressions (regex) to enforce specific tokenization rules.

### Understanding Regex in Tokenization

Regex patterns play a crucial role in defining how text should be tokenized. They allow for the specification of rules that determine which parts of the text should not be merged, ensuring more precise tokenization.

<img src="frames/frame_3595.jpg" alt=" "They create a regex pattern here that looks very complicated, and we're going to go through it in a bit."" width="450"/>

#### The Power of the `regex` Package

It's important to distinguish between the standard Python `re` module and the `regex` package. The latter is an extension that offers more capabilities and is essential for complex tokenization tasks.

[Jump to this part of the video: 01:00:18](https://www.youtube.com/watch?v=zduSFxRajkE&t=3618s)

### Breaking Down the Regex Pattern

The regex pattern used in advanced tokenization can seem daunting at first glance. However, understanding its components reveals how it effectively separates text into tokens.

<img src="frames/frame_3631.jpg" alt=" "So let's take a look at this pattern and what it's doing and why this is actually doing the separation that they are looking for."" width="450"/>

#### How the Pattern Works

The pattern is designed to match specific sequences in the text, using a combination of raw strings and logical ORs (`|`). This approach allows it to identify and separate different elements within the text, such as words from spaces or punctuation.

[Jump to this part of the video: 01:00:46](https://www.youtube.com/watch?v=zduSFxRajkE&t=3646s)

### Practical Application: Tokenizing a String

To see the regex pattern in action, we apply it to an example string. This demonstrates how the pattern matches different parts of the text, effectively breaking it down into tokens.

<img src="frames/frame_3646.jpg" alt=" "So what exactly is this doing? Well, re.findall will take this pattern and try to match it against this string."" width="450"/>

#### The Outcome of Tokenization

By applying the regex pattern to a sample string, we can observe how it successfully identifies and separates words. This process is crucial for feeding accurately tokenized text into an LLM like GPT-2.

[Jump to this part of the video: 01:02:54](https://www.youtube.com/watch?v=zduSFxRajkE&t=3774s)

Advanced Tokenization Techniques
--------------------------------

Tokenization is not just about converting text into tokens; it's about intelligently splitting text to ensure that the model processes it in the most efficient and accurate way possible. This section delves into the sophisticated methods used to split text before it undergoes tokenization, highlighting the importance of this preliminary step.

<img src="frames/frame_3781.jpg" alt=" Now, what is this doing and why is this important?" width="450"/>

### Splitting Text for Better Tokenization

The process begins by dividing the text into a list of smaller text elements. Each element is then tokenized independently, and the resulting sequences of tokens are concatenated. This method ensures that certain combinations of characters, such as a letter followed by a space, are never merged, maintaining the integrity of the tokenization process.

[Jump to this part of the video: 00:59:49](https://www.youtube.com/watch?v=zduSFxRajkE&t=3589s)

### Utilizing Regex for Text Segmentation

Regular expressions (regex) play a crucial role in segmenting text into manageable pieces. By applying specific regex patterns, the tokenizer can separate letters, numbers, and punctuation, preventing unwanted merges during the tokenization process. This technique is essential for maintaining the granularity of the tokenized text.

<img src="frames/frame_3874.jpg" alt=" So basically using this regex pattern to chunk up the text is just one way of enforcing that some merges are not to happen" width="450"/>

#### Handling Numeric Characters

The tokenizer uses the `\p{n}` pattern to identify numeric characters across various scripts, ensuring that numbers are treated as distinct entities. This separation is vital for correctly processing text that includes both letters and numbers.

[Jump to this part of the video: 01:04:57](https://www.youtube.com/watch?v=zduSFxRajkE&t=3897s)

#### Dealing with Apostrophes

Apostrophes present a unique challenge in tokenization. The tokenizer includes specific patterns to identify common apostrophes, separating them from adjacent letters. However, issues arise with Unicode apostrophes, which are not always correctly segmented, demonstrating the complexities of tokenization.

<img src="frames/frame_3926.jpg" alt=" Let's see how these apostrophes work" width="450"/>

This advanced approach to tokenization, involving text splitting and the use of regex, underscores the intricate processes required to prepare text for large language models. By understanding these techniques, developers can better manage the challenges of tokenization, ensuring that their models process text as accurately as possible.

Exploring the Intricacies of Tokenization in GPT Models
=======================================================

Tokenization Challenges and Solutions
-------------------------------------

Tokenization in GPT models involves complex patterns and rules to effectively separate text into tokens. This process is crucial for the model's understanding and processing of text data. One notable challenge is handling apostrophes in different cases, which can lead to inconsistent tokenization.

<img src="frames/frame_3989.jpg" alt=" And so it's basically hard-coded for this specific kind of apostrophe, and otherwise they become completely separate tokens." width="450"/>

GPT-2 Documentation Insights
----------------------------

The GPT-2 documentation highlights the importance of considering case sensitivity in tokenization patterns. The absence of `re.ignorecase` results in different tokenization outcomes for uppercase and lowercase versions of words with apostrophes, demonstrating the nuanced nature of tokenization rules.

[Jump to this part of the video: 01:06:42](https://www.youtube.com/watch?v=zduSFxRajkE&t=4002s)

Language-Specific Tokenization Issues
-------------------------------------

Tokenization rules can also be language-specific, leading to inconsistencies when dealing with languages that use apostrophes differently. This highlights the challenge of creating a universally effective tokenizer across different languages.

<img src="frames/frame_4002.jpg" alt=" In addition to this, you can go to the GPT-2 docs, and here when they define the pattern, they say, should have added re.ignore case, so BP mergers can happen for capitalized versions of contractions." width="450"/>

Regex Patterns in Tokenization
------------------------------

The tokenizer uses regex patterns to match and separate punctuation, numbers, and letters. This method ensures that punctuation is correctly identified and separated, showcasing the tokenizer's ability to handle various text elements.

<img src="frames/frame_4069.jpg" alt=" And what this is saying is, again, optional space followed by something that is not a letter, number, or a space, and one or more of that." width="450"/>

Handling Whitespace in Tokenization
-----------------------------------

GPT-2's tokenizer employs a unique approach to whitespace, using negative lookahead assertions in regex to manage spaces efficiently. This technique ensures that common tokens with spaces are consistently tokenized, maintaining the model's performance.

<img src="frames/frame_4107.jpg" alt=" And finally, this is also a little bit confusing. So this is matching whitespace, but this is using a negative lookahead assertion in regex." width="450"/>

Real-World Tokenization Example
-------------------------------

A practical example of tokenization in action is demonstrated with a piece of Python code. The tokenizer's ability to split text into distinct elements, without merging spaces, showcases its effectiveness in handling complex text structures.

<img src="frames/frame_4194.jpg" alt=" I wanted to show one more real world example here. So if we have this string, which is a piece of Python code, and then we try to split it up, then this is the kind of output we get." width="450"/>

Understanding OpenAI's Tokenization Approach
--------------------------------------------

OpenAI's methodology for training the GPT-2 tokenizer remains partially undisclosed, with only the inference code released. This leaves some aspects of the tokenizer's training process a mystery, highlighting the complexity of developing effective tokenization strategies.

<img src="frames/frame_4267.jpg" alt=" Now, the training code for the GPT-2 tokenizer was never released, so all we have is the code that I've already shown you, but this code here that they've released is only the inference code for the tokens." width="450"/>

Introducing the TickToken Library
---------------------------------

The TickToken library is OpenAI's official tool for tokenization inference. It simplifies the process of tokenizing text for GPT models, including the latest GPT-4, which introduces changes in whitespace handling and regex patterns for improved tokenization.

<img src="frames/frame_4301.jpg" alt=" Next, I wanted to introduce you to the TickToken library from OpenAI, which is the official library for tokenization from OpenAI." width="450"/>

GPT-4 Tokenizer Enhancements
----------------------------

GPT-4 introduces significant changes in its tokenizer, including a revised regex pattern and the handling of special tokens. These modifications aim to enhance the model's ability to process and understand text more effectively.

<img src="frames/frame_4398.jpg" alt=" And then if you scroll down to CL100K, this is the GPT-4 tokenizer, you see that the pattern has changed." width="450"/>

## Enhancements in GPT-4 Tokenization
### Case Sensitivity and Apostrophe Handling
GPT-4 introduces several changes to improve tokenization, including case-insensitive matching for contractions such as 's, 'd, 'm, etc., ensuring both lowercase and uppercase versions are recognized equally.

<img src="frames/frame_4436.jpg" alt=" And so the comment that we saw earlier on, oh, we should have used re.uppercase, basically we're now going to be matching these, apostrophe s, apostrophe d, apostrophe m, etc." width="450"/>

### Number Tokenization Limitations
Another notable adjustment is the limitation on number tokenization, where sequences of more than three digits are not merged, aiming to prevent excessively long number sequences in tokens.

<img src="frames/frame_4458.jpg" alt=" And then one more thing here is you will notice that when they match the numbers, they only match one to three numbers." width="450"/>

### Vocabulary Size Expansion
The vocabulary size has significantly increased from approximately 50k to around 100k, enhancing the model's ability to understand and generate a wider array of text.

## Deep Dive into GPT-2's Encoder.py
### Understanding OpenAI's Tokenizer Files
The `encoder.json` and `vocab.bpe` files are crucial for the tokenizer's functionality, with the former acting as the vocabulary and the latter detailing the merges.

[Jump to this part of the video: 01:15:13](https://www.youtube.com/watch?v=zduSFxRajkE&t=4513s)

### The Byte Encoder and Decoder
OpenAI's implementation includes a byte encoder and decoder, which, despite being an additional layer to the tokenizer, are not extensively covered due to their straightforward nature.

<img src="frames/frame_4613.jpg" alt=" And this is actually unfortunately just kind of a spurious implementation detail." width="450"/>

### The Core of BPE Tokenization
The BPE (Byte Pair Encoding) function is the heart of the tokenizer, identifying and merging bigrams in a loop until no further merges are possible, mirroring the process we've previously discussed.

<img src="frames/frame_4659.jpg" alt=" And you should recognize this loop here, which is very similar to our own while loop," width="450"/>

## Special Tokens in Tokenization
### The Role of Special Tokens
Special tokens, such as the end-of-text token, are used to delimit documents or sections within the data, aiding the model in understanding when one segment ends and another begins.

<img src="frames/frame_4797.jpg" alt=" So when we're creating the training data, we have all these documents, and we tokenize them and get a stream of tokens." width="450"/>

### Implementing Special Tokens
The implementation of special tokens varies, with some being handled outside the typical BPE algorithm, allowing for the addition of arbitrary tokens to the tokenizer's vocabulary.

[Jump to this part of the video: 01:21:39](https://www.youtube.com/watch?v=zduSFxRajkE&t=4899s)

### Extending Tokenizers with Special Tokens
The ability to extend tokenizers with new special tokens is highlighted, showcasing the flexibility in customizing tokenization for specific needs or applications.

<img src="frames/frame_5015.jpg" alt=" Now we can also go back to this file, which we looked at previously." width="450"/>

### GPT-4's Additional Special Tokens
GPT-4 introduces new special tokens, including FIM (fill in the middle) tokens, to facilitate more complex tokenization strategies, reflecting advancements in tokenizer design.

[Jump to this part of the video: 01:24:04](https://www.youtube.com/watch?v=zduSFxRajkE&t=5044s)

Adding Special Tokens to Your Model
-----------------------------------

Incorporating special tokens into a language model requires careful adjustments, often referred to as "model surgery." This process is essential when fine-tuning a model for specific tasks, such as adapting a base model into a chat model like ChatGPT. Adding a special token involves extending the embedding matrix and the final classifier layer to accommodate the new token.

- **Embedding Matrix Extension**: A new row is added to the embedding matrix for each special token, typically initialized with small random numbers.
- **Classifier Layer Adjustment**: The projection into the classifier at the end of the transformer model must also be extended to include the new token.

This operation is common in model fine-tuning and is crucial for those looking to customize their models for specific applications.

<img src="frames/frame_5065.jpg" alt=" And then there's one additional SERP token here. So that's that encoding as well." width="450"/>

Building Your Own GPT-4 Tokenizer
---------------------------------

To build a GPT-4 tokenizer, you can follow a structured approach, breaking down the task into manageable steps. The MinBPE repository and accompanying exercise progression provide a roadmap for developing a tokenizer capable of encoding and decoding strings to and from tokens. This process is vital for anyone looking to understand or implement their own version of a tokenizer for large language models.

- **MinBPE Repository**: Offers code and tests that can be referenced when building your tokenizer.
- **Exercise Progression**: A guide divided into four steps, designed to help you build up to a fully functional GPT-4 tokenizer.

Following these resources closely and experimenting with the provided code can lead to a deeper understanding of tokenization processes and their implementation.

[Jump to this part of the video: 01:25:43](https://www.youtube.com/watch?v=zduSFxRajkE&t=5143s)

Visualizing Token Merges and Vocabulary Training
------------------------------------------------

Training your tokenizer involves understanding how tokens are merged and visualizing the vocabulary that results from this process. The MinBPE repository showcases the initial merges and the order in which they occur, providing insights into the tokenizer's training process. For example, the first merge performed by GPT-4 was combining two spaces into a single token. This visualization helps in comprehending how token vocabularies evolve during training.

- **GPT-4 Merges Visualization**: Demonstrates the sequence of token merges during the training of GPT-4, offering a glimpse into the model's learning process.
- **Token Vocabulary Training**: By training your tokenizer, you can develop a custom token vocabulary, as shown in the MinBPE code examples.

This aspect of tokenizer development is crucial for those aiming to tailor their models to specific datasets or applications.

<img src="frames/frame_5225.jpg" alt=" So here's some of the code inside MinBPE, shows the token vocabularies that you might obtain." width="450"/>

Exploring Advanced Tokenization Techniques
------------------------------------------

In the realm of large language models (LLMs), understanding the nuances of tokenization techniques is crucial for optimizing model performance. This section delves into the comparison between different tokenization strategies employed by state-of-the-art models like GPT-4 and the tools used for tokenization, such as TICToken and SentencePiece.

### Comparing Tokenization in GPT-4 and Custom Models

Tokenization plays a pivotal role in how LLMs interpret and process text. A fascinating observation is the way GPT-4 and custom models handle tokenization differently, primarily due to the variance in their training datasets.

<img src="frames/frame_5282.jpg" alt=" And so as an example, here GPT-4 merged IN to become IN" width="450"/>

For instance, GPT-4's tokenizer might merge certain tokens differently compared to a tokenizer trained on a distinct dataset, like Wikipedia pages. This difference underscores the impact of the training set's nature on the tokenizer's behavior.

[Jump to this part of the video: 01:28:02](https://www.youtube.com/watch?v=zduSFxRajkE&t=5282s)

### Introduction to SentencePiece

Moving beyond the traditional tokenization approach, we encounter SentencePiece, a versatile library favored for its efficiency in both training and inference phases of LLMs.

- **Efficiency Across Phases**: SentencePiece stands out by supporting a wide array of algorithms, including the byte-pair encoding (BPE) algorithm, making it a go-to choice for many language models.

[Jump to this part of the video: 01:28:57](https://www.youtube.com/watch?v=zduSFxRajkE&t=5337s)

- **Unique Approach to Tokenization**: Unlike TICToken, which operates on bytes post UTF-8 encoding, SentencePiece directly manipulates the code points in the string. This fundamental difference in approach allows SentencePiece to offer a unique perspective on tokenization.

<img src="frames/frame_5378.jpg" alt=" So in the case of TICToken, we first take our code points in the string, we encode them using UTF-8 to bytes, and then we're merging bytes" width="450"/>

### The Subtleties of SentencePiece's Methodology

SentencePiece's methodology is nuanced, focusing on the code points from the training set and merging them. This process is distinct from the byte-level merging seen in other tokenization methods.

- **Handling of Rare Code Points**: A notable feature of SentencePiece is its handling of rare code points. Depending on the character coverage hyperparameter, rare code points might be mapped to a special unknown token or, with the byte fallback option, encoded into UTF-8 bytes and then tokenized.

[Jump to this part of the video: 01:30:05](https://www.youtube.com/watch?v=zduSFxRajkE&t=5405s)

This nuanced approach to tokenization, focusing on code points and their rarity, highlights the adaptability and depth of SentencePiece in managing the complexities of language model training.

### Personal Insights on Tokenization Methods

The comparison between TICToken and SentencePiece reveals a significant, albeit subtle, difference in their tokenization philosophies. While TICToken's method is perceived as cleaner due to its straightforward byte merging, SentencePiece offers a more intricate strategy by directly dealing with code points and providing fallback mechanisms for rare instances.

<img src="frames/frame_5446.jpg" alt=" Personally, I find the TICToken way significantly cleaner, but it's kind of like a subtle but pretty major difference between the way they approach tokenization" width="450"/>

Understanding these advanced tokenization techniques is essential for anyone looking to delve deeper into the workings of LLMs and optimize their models for better performance and efficiency.

Exploring SentencePiece for Tokenization
-----------------------------------------

In our journey to understand tokenization in large language models, we delve into the use of SentencePiece, a robust tool designed to handle the complexities of tokenization with a wide array of configurations. SentencePiece stands out due to its flexibility and ability to manage diverse linguistic data, making it a valuable asset in the tokenization process.

<img src="frames/frame_5461.jpg" alt=" This is how we can import SentencePiece" width="450"/>

Creating a Toy Dataset for SentencePiece
----------------------------------------

To demonstrate SentencePiece's capabilities, we create a simple toy dataset contained within a "toy.txt" file. This practical example helps illustrate how SentencePiece operates, emphasizing its preference for file-based input, which is crucial for its functionality.

[Jump to this part of the video: 01:31:04](https://www.youtube.com/watch?v=zduSFxRajkE&t=5464s)

Understanding SentencePiece's Configuration Complexity
-------------------------------------------------------

SentencePiece's vast range of options and configurations can be overwhelming, attributed to its long history and aim to accommodate a broad spectrum of use cases. While many options may seem irrelevant for specific tasks, understanding these configurations is key to leveraging SentencePiece effectively.

<img src="frames/frame_5482.jpg" alt=" And the reason this is so is because SentencePiece has been around, I think, for a while" width="450"/>

Configuring SentencePiece for LLM Tokenization
-----------------------------------------------

Our goal is to configure SentencePiece in a manner that aligns with the tokenization approach used by Llama2, as trained by Meta. By examining and replicating the relevant options from the tokenizer.model file released by Meta, we aim to achieve a setup that mirrors their methodology, focusing on the BP (byte-pair) encoding algorithm and a vocabulary size of 400.

[Jump to this part of the video: 01:32:44](https://www.youtube.com/watch?v=zduSFxRajkE&t=5564s)

The Role of Normalization in Tokenization
-----------------------------------------

Normalization plays a significant role in traditional natural language processing tasks, such as machine translation and text classification. However, in the context of language models, the preference often leans towards minimal interference with the raw data, aiming to preserve its original form as much as possible.

<img src="frames/frame_5589.jpg" alt=" Normalization used to be very prevalent, I would say, before LLMs in natural language processing" width="450"/>

SentencePiece and the Concept of Sentences
------------------------------------------

SentencePiece introduces the concept of sentences as individual training examples, a notion that may not align perfectly with the requirements of large language models. This distinction highlights the challenges in defining what constitutes a sentence, especially when considering the nuances present in different languages and datasets.

[Jump to this part of the video: 01:33:45](https://www.youtube.com/watch?v=zduSFxRajkE&t=5625s)

Exploring Advanced Tokenization Techniques
------------------------------------------

Tokenization is not just about splitting text into manageable pieces; it involves intricate rules and considerations, especially when dealing with large language models. This section delves into the complexities of tokenization, highlighting the challenges of handling rare word characters, digits, whitespace, and the implementation of merge rules.

<img src="frames/frame_5683.jpg" alt=" It has a lot of treatment around rare word characters, and when I say word, I mean code points" width="450"/>

Understanding Special Tokens and Vocabulary Creation
----------------------------------------------------

Special tokens play a crucial role in tokenization, serving as markers for the beginning and end of sentences, padding, and undefined tokens (UNCTOKEN). These tokens are essential for training language models effectively. The process of creating a vocabulary file and a model file during training is demonstrated, showcasing how SentenceBees handles vocabulary with a focus on special tokens, byte tokens, merge tokens, and individual code point tokens.

[Jump to this part of the video: 01:35:25](https://www.youtube.com/watch?v=zduSFxRajkE&t=5725s)

Byte Tokens and Merge Tokens: A Closer Look
--------------------------------------------

Byte tokens are a fallback mechanism in tokenization, ensuring that every possible byte (character) is represented in the vocabulary. This section explains the significance of byte tokens and how they are incorporated into the vocabulary, followed by an exploration of merge tokens, which represent parent nodes in the tokenization process.

<img src="frames/frame_5781.jpg" alt=" So here we saw that BYTE fallback in Llama was turned on, so it's true" width="450"/>

Decoding and Understanding the Vocabulary Structure
---------------------------------------------------

The structure of the vocabulary in SentenceBees is meticulously organized, starting with special tokens, followed by byte tokens, merge tokens, and finally, individual code point tokens. This organization reflects the prioritization of tokens, from the most essential to the rarest. The section also discusses how rare code points are handled, emphasizing the model's approach to maintaining a manageable and efficient vocabulary.

[Jump to this part of the video: 01:37:29](https://www.youtube.com/watch?v=zduSFxRajkE&t=5849s)

Encoding and Decoding Tokens: Practical Applications
----------------------------------------------------

Once a vocabulary is established, encoding text into token IDs and decoding them back into text are crucial capabilities of tokenization systems. This part of the guide provides insights into how SentenceBees performs encoding and decoding, illustrating the practical applications of these processes in working with large language models.

<img src="frames/frame_5865.jpg" alt=" Once we have a vocabulary, we can encode into IDs, and we can sort of get a list" width="450"/>

Exploring Token IDs and Unknown Tokens
--------------------------------------

In our journey through tokenization, we encounter a fascinating aspect when dealing with characters outside the training set, such as Korean characters in this example. These characters are not recognized by the model, leading to the use of unknown tokens (UNK) for representation. However, with byte fallback enabled, the model cleverly resorts to UTF-8 encoding to represent these unfamiliar bytes.

<img src="frames/frame_5880.jpg" alt=" So let's take a look at what happened here. Hello, space, annyeonghaseyo. So these are the token IDs we got back." width="450"/>

Understanding Byte Fallback Mechanism
-------------------------------------

Byte fallback is a crucial feature that allows the model to handle characters it hasn't encountered during training. By encoding these characters in UTF-8, the model can still process and represent them, albeit in a different form. This mechanism ensures that even unknown characters can be fed into the model, maintaining the flow of information.

[Jump to this part of the video: 01:38:33](https://www.youtube.com/watch?v=zduSFxRajkE&t=5913s)

The Impact of Disabling Byte Fallback
-------------------------------------

Disabling byte fallback leads to a significant change in how the model processes text. Without byte fallback, unknown characters are simply marked as UNK, significantly reducing the model's ability to represent and understand the input. This example highlights the importance of byte fallback in maintaining a rich and nuanced representation of text data.

<img src="frames/frame_5955.jpg" alt=" So the first thing that happened is all the byte tokens disappeared, right?" width="450"/>

Decoding Tokens and Visual Representation
-----------------------------------------

An interesting observation is made regarding the visual representation of spaces in the decoded tokens. The model appears to convert whitespace into bold underscore characters for visualization purposes. This peculiar choice might be aimed at making the spaces more noticeable during analysis.

[Jump to this part of the video: 01:40:15](https://www.youtube.com/watch?v=zduSFxRajkE&t=6015s)

Exploring the Quirks of Tokenization with SentencePiece
--------------------------------------------------------

Tokenization is a pivotal step in preparing text for processing by large language models (LLMs). It involves converting strings of text into manageable units, known as tokens, which the model can interpret. This section delves into the intricacies of tokenization using SentencePiece, highlighting its unique features and potential pitfalls.

### The Mystery of the Leading Space in Tokenization

One peculiar aspect of SentencePiece's tokenization process is the addition of an extra space at the beginning of tokens. This is controlled by the `Add dummy prefix` option, which is set to true by default.

<img src="frames/frame_6035.jpg" alt=" Why do we have an extra space in the front of hello? Where is this coming from?" width="450"/>

The rationale behind this is to standardize the treatment of words at the beginning of sentences and within sentences. By adding a dummy space, SentencePiece aims to make tokens like "world" and " world" identical in the eyes of the LLM, facilitating its learning process.

[Jump to this part of the video: 01:40:55](https://www.youtube.com/watch?v=zduSFxRajkE&t=6055s)

### SentencePiece's Approach to Preprocessing

SentencePiece employs several preprocessing options to enhance the tokenization process. One notable feature is its use of a dummy prefix to unify the representation of words, regardless of their position in a sentence. This preprocessing step is crucial for models like Lama 2, which also utilize this option for consistent tokenization.

<img src="frames/frame_6128.jpg" alt=" And that's, I think, everything that I want to say from my preview of sentence piece and how it is different" width="450"/>

### The Intricacies of SentencePiece Configuration

The configuration of SentencePiece is intricate, with various settings that influence its behavior. For those looking to replicate the tokenization process of the Meta Lama 2 model, understanding these settings is essential. The raw protocol buffer representation of the tokenizer provides a blueprint for configuring SentencePiece accordingly.

[Jump to this part of the video: 01:42:20](https://www.youtube.com/watch?v=zduSFxRajkE&t=6140s)

### Summary and Reflections on SentencePiece

In summary, SentencePiece is a widely used tool in the industry for its efficiency in both training and inference phases of tokenization. However, it comes with its share of complexities and "foot guns," such as the handling of sentence lengths and the necessity of an UNCTOKEN. Additionally, its documentation leaves much to be desired, posing challenges for users seeking to fully leverage its capabilities.

<img src="frames/frame_6156.jpg" alt=" And yeah, I think that's it for this section" width="450"/>

This exploration of SentencePiece's tokenization process sheds light on the nuanced considerations necessary for effective text processing in LLMs. Despite its quirks, SentencePiece remains a valuable tool for developers and researchers working with language models.

Exploring the Complexities of Tokenization in Large Language Models
====================================================================

### Understanding Tokenization and Its Challenges

Tokenization is a fundamental process in the operation of large language models (LLMs), involving the conversion of text data into manageable units known as tokens. This process, while essential, presents various complexities and considerations that significantly impact the performance and functionality of LLMs.

### The Importance of a Well-Designed Tokenizer

A well-designed tokenizer is crucial for the effective training and operation of LLMs. The choice of tokenizer and the decisions made during its design can have profound implications on the model's ability to understand and generate text.

<img src="frames/frame_6193.jpg" alt=" so it took me a lot of time working with this myself and just visualizing things and trying to really understand what is happening here because the documentation unfortunately is, in my opinion, not super amazing" width="450"/>

### Vocab Size Considerations in Model Architecture

When designing the vocab size for an LLM, several factors must be considered to balance computational efficiency with the model's ability to accurately represent and process text data.

- **Impact on Embedding Tables and Linear Layers**: As vocab size increases, both the token embedding table and the LM head layer's linear layer expand, requiring more computation and potentially leading to under-trained parameters due to the rarity of token occurrences in training data.

[Jump to this part of the video: 01:43:40](https://www.youtube.com/watch?v=zduSFxRajkE&t=6220s)

- **Sequence Length and Information Density**: Larger vocab sizes can lead to shorter sequences by compressing more information into each token. While this can enhance the model's ability to attend to larger contexts, it may also result in the model having insufficient capacity to process densely packed information within tokens effectively.

### Extending Vocab Size in Pre-Trained Models

Extending the vocab size of a pre-trained model involves adding new tokens to the tokenizer and adjusting the model's architecture to accommodate these additions. This process, often referred to as model surgery, requires careful consideration to ensure the new tokens are effectively integrated into the model without compromising its existing capabilities.

[Jump to this part of the video: 01:48:15](https://www.youtube.com/watch?v=zduSFxRajkE&t=6495s)

### Exploring New Frontiers: Beyond Text Tokenization

The versatility of transformers is not limited to processing text; they can be adapted to handle various input modalities, including images, videos, and audio. This adaptability is achieved by tokenizing these different types of data into formats that the transformer can process, opening up new avenues for multimodal applications.

[Jump to this part of the video: 01:50:06](https://www.youtube.com/watch?v=zduSFxRajkE&t=6606s)

### Addressing Tokenization-Related Challenges in LLMs

The process of tokenization, while essential, introduces specific challenges that can affect an LLM's performance, particularly in tasks related to spelling and handling of complex tokens.

- **Case Study: Spelling Challenges with Complex Tokens**: An examination of the token ".defaultstyle" from the GPT-4 vocabulary illustrates how the aggregation of multiple characters into a single token can hinder the model's ability to perform spelling-related tasks accurately.

[Jump to this part of the video: 01:51:59](https://www.youtube.com/watch?v=zduSFxRajkE&t=6719s)

This exploration of tokenization and its implications for LLMs underscores the importance of thoughtful tokenizer design and the ongoing exploration of techniques to enhance the versatility and effectiveness of these models in processing diverse data types.

Exploring Character-Level Tasks with GPT-4
------------------------------------------

In our journey to understand the intricacies of large language models (LLMs), we encounter various tasks that highlight the importance and challenges of tokenization. A fascinating example involves asking GPT-4 to reverse a string, `.defaultstyle`. Initially, GPT-4 attempts to use a code interpreter, but upon simplification of the task—breaking it down into smaller, more manageable steps—the model successfully reverses the string.

<img src="frames/frame_6792.jpg" alt=" So for example, here I asked GPT-4 to reverse the string .defaultstyle, and it tried to use a code interpreter" width="450"/>

This experiment underscores the hypothesis that tokenization plays a crucial role in how LLMs process and understand tasks, especially when dealing with character-level data.

[Jump to this part of the video: 01:53:05](https://www.youtube.com/watch?v=zduSFxRajkE&t=6785s)

Challenges in Non-English Language Processing
---------------------------------------------

LLMs often exhibit reduced performance in non-English languages, a phenomenon partly attributed to the models encountering less non-English data during training. However, a significant factor is the tokenizer's insufficient training on non-English data, leading to inefficiencies in token representation.

For instance, the English greeting "hello, how are you" translates into significantly more tokens in another language, illustrating the tokenizer's inefficiency and the resultant data bloat in non-English languages.

<img src="frames/frame_6867.jpg" alt=" And I briefly covered this already, but basically it's not only that the language model sees less non-English data during training" width="450"/>

This inefficiency not only affects the model's performance but also its ability to process and understand non-English languages effectively.

Tokenization and Arithmetic in LLMs
-----------------------------------

One of the intriguing aspects of LLMs is their handling of arithmetic operations, which is directly influenced by the tokenization of numbers. The process is far from straightforward, with numbers being tokenized in an arbitrary manner, affecting the model's ability to perform simple arithmetic accurately.

<img src="frames/frame_6914.jpg" alt=" That has to do with the tokenization of numbers" width="450"/>

A detailed exploration of integer tokenization reveals the randomness in how numbers are broken down into tokens, posing a significant challenge for LLMs in arithmetic tasks.

[Jump to this part of the video: 01:55:42](https://www.youtube.com/watch?v=zduSFxRajkE&t=6942s)

Improving Python Code Understanding in GPT Models
--------------------------------------------------

The tokenization process also impacts the model's efficiency in understanding and processing Python code. GPT-2, for instance, struggled with encoding spaces in Python code, treating each space as an individual token and severely limiting the model's context length.

<img src="frames/frame_7025.jpg" alt=" the encoding efficiency of the tokenizer for handling spaces in Python is terrible" width="450"/>

This issue, identified as a tokenization bug, was addressed in GPT-4, highlighting the continuous efforts to enhance LLMs' understanding and processing capabilities across different languages and data types.

[Jump to this part of the video: 01:14:53](https://www.youtube.com/watch?v=zduSFxRajkE&t=4493s)

Exploring Special Tokens and Trailing Whitespace in LLMs
========================================================

### Handling Special Tokens: A Case Study with "end of text"
------------------------------------------------------------

Large Language Models (LLMs) like GPT-4 exhibit peculiar behaviors when encountering special tokens within user inputs. An intriguing example is the model's response to the string "end of text." Despite explicit instructions to print this string, GPT-4 hesitates, questioning the specificity of the request. This incident highlights a potential parsing issue, where "end of text" might be recognized as a special token rather than a sequence of individual characters. Such anomalies suggest that the handling of special tokens by LLMs could serve as an attack surface, offering a method to confuse or manipulate model outputs.

<img src="frames/frame_7042.jpg" alt=" My LLM abruptly halts when it sees the string end of text" width="450"/>

[Jump to this part of the video: 01:57:28](https://www.youtube.com/watch?v=zduSFxRajkE&t=7048s)

### The Trailing Whitespace Issue: Impact on Tokenization and Model Performance
-------------------------------------------------------------------------------

The behavior of LLMs is also significantly influenced by the presence of trailing whitespace in inputs. This is exemplified by an experiment using GPT 3.5 Turbo Instruct, where adding a space after a tagline prompt resulted in a warning about potential performance degradation. The warning sheds light on how LLMs tokenize inputs, revealing that trailing spaces are tokenized separately, potentially disrupting the expected token sequence. This scenario underscores the importance of understanding tokenization nuances, as even seemingly minor details like trailing whitespace can lead to out-of-distribution issues, affecting model predictions and performance.

<img src="frames/frame_7134.jpg" alt=" So if you come to Playground, and we come here to GPT 3.5 Turbo Instruct" width="450"/>

[Jump to this part of the video: 01:59:31](https://www.youtube.com/watch?v=zduSFxRajkE&t=7171s)

### Tokenization and Its Quirks: Beyond Characters and Words
------------------------------------------------------------

The foundational concept of tokenization in LLMs extends beyond simple character or word recognition. Tokens represent chunks of text that the model perceives as its basic units of understanding. However, the tokenization process can introduce peculiarities, such as the unexpected handling of sequences like "default cell style" or the misinterpretation of special characters and whitespace. These examples highlight the complexity of tokenization and its critical role in shaping LLM behavior, emphasizing the need for careful consideration of input formatting to ensure optimal model performance.

<img src="frames/frame_7308.jpg" alt=" Let's go back to our default cell style" width="450"/>

[Jump to this part of the video: 02:01:48](https://www.youtube.com/watch?v=zduSFxRajkE&t=7308s)

Exploring Unstable Tokens in Language Models
--------------------------------------------

Language models can sometimes exhibit unexpected and erratic behaviors when encountering certain inputs. This phenomenon is often due to what are termed "unstable tokens," which can lead to a range of issues from the model predicting end-of-text prematurely to flagging inputs as policy violations.

<img src="frames/frame_7373.jpg" alt=" This is giving it brain damage. It's never seen this before. It's shocked, and it's predicting end of text or something." width="450"/>

### The Challenge of Partial Tokens

Partial tokens present a significant challenge in language processing. These occur when the model encounters fragments of tokens or incomplete character sequences that it fails to recognize based on its training data. This unfamiliarity can cause the model to react unpredictably, a situation colloquially described as the model feeling "jank."

[Jump to this part of the video: 02:03:09](https://www.youtube.com/watch?v=zduSFxRajkE&t=7389s)

### Unstable Tokens: A Deep Dive

The concept of unstable tokens is not widely documented, yet it plays a crucial role in understanding model behavior. By examining the codebase of tokenization tools, such as the Tuk token repository, one can find extensive handling for these unstable tokens, indicating their significance in model performance and output reliability.

[Jump to this part of the video: 02:03:39](https://www.youtube.com/watch?v=zduSFxRajkE&t=7419s)

### Advanced Tokenization Techniques

Addressing the issue of unstable tokens requires sophisticated tokenization strategies. Instead of merely appending the next full token after a partial token sequence, a more nuanced approach involves considering a range of potential characters that, upon re-tokenization, would have a high probability of being the correct continuation. This method, however, is complex and highlights the intricate challenges inherent in tokenization.

[Jump to this part of the video: 02:04:39](https://www.youtube.com/watch?v=zduSFxRajkE&t=7479s)

The Curious Case of Solid Gold Magikarp
----------------------------------------

Among the most intriguing examples of unstable tokens is the "solid gold Magikarp," a term that has gained notoriety within the language model community. This example illustrates how certain tokens, when clustered based on their embedding representations, can exhibit bizarre and unexpected behaviors.

### Unraveling the Mystery of Odd Token Clusters

A detailed analysis of token embeddings revealed clusters of seemingly unrelated and peculiar tokens, including the infamous solid gold Magikarp. This discovery raises questions about the origins of these tokens and their impact on model behavior.

<img src="frames/frame_7500.jpg" alt=" My favorite one by far is this solid gold Magikarp." width="450"/>

### Trigger Words and Model Behavior

The presence of specific trigger words, such as solid gold Magikarp, can cause language models to exhibit a wide array of strange responses. These range from evasion and hallucinations to outright insults, highlighting the unpredictable nature of model reactions to certain inputs.

[Jump to this part of the video: 02:06:13](https://www.youtube.com/watch?v=zduSFxRajkE&t=7573s)

### Investigating the Impact of Unstable Tokens

The phenomenon of unstable tokens and their associated trigger words underscores the complexity of language model behavior. It reveals the need for further research and development to mitigate these issues, ensuring models operate within expected safety guidelines and alignment principles.

[Jump to this part of the video: 02:07:02](https://www.youtube.com/watch?v=zduSFxRajkE&t=7622s)

Exploring the Quirks of Tokenization in Language Models
-------------------------------------------------------

Tokenization plays a pivotal role in the functionality of large language models (LLMs), often leading to unexpected behaviors due to the nuances in its implementation. This section delves into a specific instance highlighting the importance of understanding tokenization's impact on LLMs.

### The Case of the "sold gold Magikarp"
<img src="frames/frame_347.jpg" alt=" Well this again comes down to tokenization" width="450"/>

A curious case emerged involving a Reddit user known as "sold gold Magikarp." This example illustrates how discrepancies between the tokenization dataset and the training dataset can lead to unforeseen model behavior. The tokenization process identified "sold gold Magikarp" as a frequently occurring string, assigning it a unique token within the model's vocabulary. However, this token was absent from the training data used to refine the model, resulting in an "untrained" token that, when invoked, could lead to unpredictable outcomes.

[Jump to this part of the video: 02:07:13](https://www.youtube.com/watch?v=zduSFxRajkE&t=7633s)

### Untrained Tokens and Undefined Behavior
The phenomenon of untrained tokens, akin to unallocated memory in programming, underscores the challenges in managing LLMs' vocabularies. When such tokens are activated, they extract an untrained row from the embedding table, introducing undefined behavior into the model's output. This scenario emphasizes the critical nature of aligning tokenization and training datasets to ensure model reliability.

<img src="frames/frame_7736.jpg" alt=" And then at test time, if you evoke this token, then you're basically plucking out a row of the embedding table that is completely untrained" width="450"/>

### Tokenization Efficiency Across Formats
The efficiency of tokenization can vary significantly across different data formats, impacting the model's performance and cost-effectiveness. A comparison between JSON and YAML demonstrates this variability, with YAML proving to be more token-efficient. This insight is valuable for optimizing the processing of structured data, encouraging the selection of formats that minimize token usage and, by extension, computational resources.

[Jump to this part of the video: 02:09:37](https://www.youtube.com/watch?v=zduSFxRajkE&t=7777s)

### Concluding Thoughts on Tokenization
Despite its complexities and potential pitfalls, tokenization remains a crucial stage in the development and operation of LLMs. The challenges it presents, from security issues to AI safety concerns, highlight the importance of thorough understanding and careful management of this process. The pursuit of more efficient and effective tokenization methods continues, with the potential to significantly enhance the performance and applicability of LLMs in various domains.

<img src="frames/frame_7824.jpg" alt=" Okay, so that concludes my fairly long video on tokenization" width="450"/>

### Recommendations for Tokenization Practices
For those working with LLMs, leveraging existing tokens and vocabularies, such as those from GPT-4, can offer efficiency benefits. Tools like TIC token provide valuable resources for inference, while custom vocabulary training may benefit from sentence piece BP, depending on specific project requirements.

[Jump to this part of the video: 02:11:04](https://www.youtube.com/watch?v=zduSFxRajkE&t=7864s)

Challenges with SentencePiece Tokenization
------------------------------------------

In the realm of tokenization for large language models, SentencePiece is a tool that comes with its own set of challenges. Despite its widespread use, there are several aspects of SentencePiece that can make it less than ideal for certain applications.

- **Byte Fallback and Unicode Code Points**: One of the primary concerns with SentencePiece is its reliance on byte fallback and the handling of Unicode code points during the Byte Pair Encoding (BPE) process. This approach can introduce complications, especially when dealing with a diverse set of characters and symbols.

<img src="frames/frame_7891.jpg" alt=" As I mentioned, I'm not a huge fan of sentence piece. I don't like its byte fallback" width="450"/>

- **Complexity and Configuration**: SentencePiece is known for its myriad of settings, which can be both a blessing and a curse. The flexibility allows for fine-tuning, but it also introduces the risk of misconfiguration, leading to potential issues such as inadvertently cropping sentences due to misunderstood parameters.

<img src="frames/frame_7901.jpg" alt=" And I think there's a lot of foot guns here" width="450"/>

- **Recommendations for Use**: For those who decide to use SentencePiece, it's advised to proceed with caution regarding its settings. One strategy is to replicate the configurations used by established projects, such as those by Meta, to minimize the risk of errors. Additionally, spending time to thoroughly understand the available hyperparameters and their implications is crucial for successful implementation.

[Jump to this part of the video: 02:11:49](https://www.youtube.com/watch?v=zduSFxRajkE&t=7909s)

Exploring Alternatives: The Promise of minBPE
---------------------------------------------

While SentencePiece has its place in the toolkit of tokenization methods, the search for more efficient and effective alternatives is ongoing. One promising direction is the development of minBPE, a method that aims to combine the benefits of token-based approaches with the efficiency of code training.

- **The Ideal of TIC Token with Training Code**: The ultimate goal in tokenization is to achieve a method that combines the granularity of token-based approaches with the adaptability of code training. This combination would offer the best of both worlds, providing a robust solution for tokenization challenges.

<img src="frames/frame_7946.jpg" alt=" And that is the ideal thing that currently does not exist" width="450"/>

- **Current State of minBPE**: As of now, minBPE represents a step towards this ideal, though it is still in the development phase and primarily implemented in Python. The anticipation for minBPE to reach its full potential is high, as it promises to bring significant improvements to the process of tokenization.

[Jump to this part of the video: 02:12:10](https://www.youtube.com/watch?v=zduSFxRajkE&t=7930s)

Looking Ahead: Future Directions in Tokenization
------------------------------------------------

The journey of refining tokenization methods is far from over. With ongoing research and development, new approaches like minBPE are on the horizon, offering hope for more efficient and effective solutions. As the field progresses, staying informed and adaptable will be key for those working with large language models.

- **Anticipation for Advanced Developments**: The possibility of more detailed and advanced discussions on tokenization in the future is open. As the technology evolves, so too will the strategies for optimizing the tokenization process.

[Jump to this part of the video: 02:12:41](https://www.youtube.com/watch?v=zduSFxRajkE&t=7961s)

- **Continued Evolution of Context Size in Models**: An interesting development in the field is the increase in context size from earlier models like GPT-1's 512 to GPT-2's 1024. This evolution highlights the ongoing efforts to enhance the capabilities of large language models through improvements in tokenization and model architecture.

[Jump to this part of the video: 02:12:58](https://www.youtube.com/watch?v=zduSFxRajkE&t=7978s)

In conclusion, while challenges with current tokenization methods like SentencePiece exist, the future holds promise for more refined and efficient approaches. Through careful consideration of settings and an eye towards emerging technologies like minBPE, the field continues to advance towards more effective solutions for processing and understanding language at scale.